In [42]:
import pandas as pd
import numpy as np
import pyodbc
import bcrypt
from faker import Faker
import random
from datetime import datetime, timedelta

In [46]:
# Create a Faker instance for generating dummy data
faker = Faker()

# Define MSSQL connection details
connection_string = 'DRIVER={SQL Server};SERVER=DESKTOP-CMTGLLQ;DATABASE=JLEARN;trusted_connection=YES'

# Connect to the database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()


In [44]:
# Function to hash passwords (simulates Next.js hashing)
def hash_password(password):
    return bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt(10)).decode('utf-8')

# Generate dummy users and insert into MSSQL
def generate_dummy_users(num_users):
    for _ in range(num_users):
        username = faker.user_name()
        full_name = faker.name()
        email = faker.unique.email()
        password_hash = hash_password("123")  # All users will have '123' as their password
        role = 'employee'
        registered_at = datetime.now()

        # Insert user into MSSQL database
        cursor.execute('''
            INSERT INTO users (UserName, FullName, Email, PasswordHash, Role, RegisteredAt)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (username, full_name, email, password_hash, role, registered_at))
        
    # Commit the transaction
    conn.commit()
    print(f'{num_users} users inserted successfully!')

# Generate 10 dummy users
generate_dummy_users(97)

# Close the connection
cursor.close()
conn.close()


97 users inserted successfully!


In [47]:
# Get existing users from the users table
cursor.execute("SELECT UserID FROM users")
users = [row[0] for row in cursor.fetchall()]

# Get existing courses from the courses table
cursor.execute("SELECT course_id FROM courses")
courses = [row[0] for row in cursor.fetchall()]

# Function to generate user_course records
def generate_user_courses(num_records):
    for _ in range(num_records):
        user_id = random.choice(users)  # Assign random user
        for _ in range(2):
            course_id = random.choice(courses)  # Assign random course
            
            status = random.choice(["assigned", "undertaken"])
            progress = 100
            
            enrolled_at = faker.date_time_between(start_date='-1y', end_date='now')
            completed_at = enrolled_at + timedelta(days=random.randint(1, 100))
            
            score = random.randint(50,100)
            
            # Insert user_course record into the database
            cursor.execute('''
                INSERT INTO user_courses (user_id, course_id, status, progress, enrolled_at, completed_at, score)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (user_id, course_id, status, progress, enrolled_at, completed_at, score))

    # Commit the transaction
    conn.commit()
    print(f'{num_records} user_course records inserted successfully!')

# Generate 50 user_course records
generate_user_courses(100)

# Close the connection
cursor.close()
conn.close()


100 user_course records inserted successfully!
